# ETL - TCC

## Com PySpark

### Carregando Bibliotecas

In [2]:
import os

os.environ["SPARK_HOME"] = "D:\spark\spark-3.2.1-bin-hadoop3.2"
os.environ['HADOOP_HOME'] = "D:\spark\spark-3.2.1-bin-hadoop3.2" 

os.environ['JAVA_HOME'] = "C:\Program Files\Java\jre1.8.0_202"

jardrv = r"C:\Program Files\MySQL\Connector J 8.0\mysql-connector-java-8.0.26.jar"

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
import random
import pandas as pd
import pyspark.pandas as ps
import pyspark.sql.functions

In [5]:
spark = SparkSession.builder.config("spark.jars", jardrv).config('spark.driver.extraClassPath', jardrv).getOrCreate()

In [31]:
sc = spark.sparkContext

# ENEM 2020

### Carregando Dados

In [6]:
enem_2020 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2020\DADOS\MICRODADOS_ENEM_2020.csv", sep=';', inferSchema=True, header=True)

In [53]:
columns=['NU_INSCRICAO', 'NU_ANO', 'TP_COR_RACA', 'TP_ESTADO_CIVIL','TP_SEXO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006']

In [54]:
enem_2020_selected = enem_2020.select(columns)

In [55]:
enem_2020_without_nan = enem_2020_selected.dropna(subset = ['NU_NOTA_CN'])

In [56]:
enem_2020_without_nan = enem_2020_without_nan.withColumn('NU_NOTA_CN',enem_2020_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2020_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2020_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2020_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2020_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2020_without_nan['Q005'].cast('int'))

### Adicionando Média das Notas

Pesquisa com Base curso de Medicina na USP

https://www.sisu.pro.br/sisu/sisu-2021-usp-2.html#:~:text=Pesos%20para%20Medicina&text=Linguagens%20e%20C%C3%B3digos%3A%202%C3%97,Matem%C3%A1tica%3A%203%C3%97500%20%3D%201.500

* Redação: 2
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 4
* Matemática: 3

In [73]:
def mean_med(df):
    df2 = df.withColumn("MEDIA_MED", (( \
        df['NU_NOTA_CN'] * 4 + \
        df['NU_NOTA_CH'] * 2 + \
        df['NU_NOTA_LC'] * 2 + \
        df['NU_NOTA_MT'] * 3 + \
        df['NU_NOTA_REDACAO'] * 2\
    ) / (4+2+2+3+2)))
    return df2

In [74]:
enem_2020_without_nan2 = mean_med(enem_2020_without_nan)

TypeError: type Column doesn't define __round__ method

Pesquisa com Base curso de Administração na Unicentro Paraná

https://www3.unicentro.br/vestibular/sisu/cursos-e-pesos/


* Redação: 1
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 2
* Matemática: 3

In [59]:
def mean_adm(df):
    df2 = df.withColumn("MEDIA_ADM", (( \
        df['NU_NOTA_CN'] * 2 + 
        df['NU_NOTA_CH'] * 2 + 
        df['NU_NOTA_LC'] * 2 + 
        df['NU_NOTA_MT'] * 3 + 
        df['NU_NOTA_REDACAO'] * 1
    ) / (2+2+2+3+1)))
    return df2

In [60]:
enem_2020_without_nan3 = mean_adm(enem_2020_without_nan2)

### Tranformando pyspark em pandas

In [47]:
count_enem_2020 = list(range(0, enem_2020_without_nan3.count()))

In [48]:
index_df = sc.parallelize(count_enem_2020).map(lambda x: (x, )).toDF(['index'])

### Inserção

In [ ]:
import mysql.connector as mysql
from mysql.connector import Error
from config import config
import csv

In [ ]:
enem_2020_without_nan.write.format("jdbc") \
    .option("url", "jdbc:mysql://----") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", 'enem_tcc') \
    .option("user", "---") \
    .option("password", "----") \
    .mode("append") \
    .save()

# OUTROS ANOS

In [6]:
enem_2019 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2019\DADOS\MICRODADOS_ENEM_2019.csv", sep=';', inferSchema=True, header=True)
enem_2018 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2018\DADOS\MICRODADOS_ENEM_2018.csv", sep=';', inferSchema=True, header=True)
enem_2017 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2017\DADOS\MICRODADOS_ENEM_2017.csv", sep=';', inferSchema=True, header=True)
enem_2016 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2016\DADOS\MICRODADOS_ENEM_2016.csv", sep=';', inferSchema=True, header=True)
enem_2013 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2013\DADOS\MICRODADOS_ENEM_2013.csv", sep=';', inferSchema=True, header=True)
enem_2010 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2010\DADOS\MICRODADOS_ENEM_2010.csv", sep=';', inferSchema=True, header=True)

In [ ]:
enem_2012 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2012\DADOS\MICRODADOS_ENEM_2012.csv", inferSchema=True, header=True)
enem_2015 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2015\DADOS\MICRODADOS_ENEM_2015.csv", inferSchema=True, header=True)
enem_2014 = spark.read.csv(r"D:\DESKTOP\Documents\TCC\base_dados\enem\microdados_enem2014\DADOS\MICRODADOS_ENEM_2014.csv", inferSchema=True, header=True)

In [ ]:
enem_2019_without_nan2 = mean_med(enem_2019_without_nan)
enem_2018_without_nan2 = mean_med(enem_2018_without_nan)
enem_2017_without_nan2 = mean_med(enem_2017_without_nan)
enem_2016_without_nan2 = mean_med(enem_2016_without_nan)
enem_2015_without_nan2 = mean_med(enem_2015_without_nan)
enem_2014_without_nan2 = mean_med(enem_2014_without_nan)
enem_2013_without_nan2 = mean_med(enem_2013_without_nan)
enem_2012_without_nan2 = mean_med(enem_2012_without_nan)
enem_2010_without_nan2 = mean_med(enem_2010_without_nan)

In [ ]:
enem_2019_without_nan3 = mean_adm(enem_2019_without_nan2)
enem_2018_without_nan3 = mean_adm(enem_2018_without_nan2)
enem_2017_without_nan3 = mean_adm(enem_2017_without_nan2)
enem_2016_without_nan3 = mean_adm(enem_2016_without_nan2)
enem_2015_without_nan3 = mean_adm(enem_2015_without_nan2)
enem_2014_without_nan3 = mean_adm(enem_2014_without_nan2)
enem_2013_without_nan3 = mean_adm(enem_2013_without_nan2)
enem_2012_without_nan3 = mean_adm(enem_2012_without_nan2)
enem_2010_without_nan3 = mean_adm(enem_2010_without_nan2)

### Selecionando Colunas Importantes

In [ ]:
columns=['NU_ANO', 'TP_COR_RACA', 'TP_ESTADO_CIVIL','TP_SEXO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006']

columns_2012=['NU_ANO', 'TP_COR_RACA', 'TP_ESTADO_CIVIL', 'TP_SEXO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

columns_2010 = ['NU_ANO', 'TP_COR_RACA', 'TP_ESTADO_CIVIL','TP_SEXO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006']

In [ ]:
enem_2014 = enem_2014.withColumnRenamed("NOTA_CN", "NU_NOTA_CN")\
                     .withColumnRenamed("NOTA_LC", "NU_NOTA_LC")\
                     .withColumnRenamed("NOTA_CH", "NU_NOTA_CH")\
                     .withColumnRenamed("NOTA_MT", "NU_NOTA_MT")

In [ ]:
enem_2013 = enem_2013.withColumnRenamed("NOTA_CN", "NU_NOTA_CN")\
                     .withColumnRenamed("NOTA_LC", "NU_NOTA_LC")\
                     .withColumnRenamed("NOTA_CH", "NU_NOTA_CH")\
                     .withColumnRenamed("NOTA_MT", "NU_NOTA_MT")

In [ ]:
enem_2012 = enem_2012.withColumnRenamed("NU_NT_CN", "NU_NOTA_CN")\
                     .withColumnRenamed("NU_NT_CH", "NU_NOTA_LC")\
                     .withColumnRenamed("NU_NT_LC", "NU_NOTA_CH")\
                     .withColumnRenamed("NU_NT_MT", "NU_NOTA_MT")

In [ ]:
enem_2010 = enem_2010.withColumnRenamed("Q01", "Q001")\
                     .withColumnRenamed("Q02", "Q002")\
                     .withColumnRenamed("Q03", "Q003")\
                     .withColumnRenamed("Q04", "Q004")\
                     .withColumnRenamed("Q05", "Q005")\
                     .withColumnRenamed("Q06", "Q006")

In [ ]:
enem_2010_selected = enem_2010.select(columns_2010)
enem_2012_selected = enem_2012.select(columns_2012)
enem_2013_selected = enem_2013.select(columns)
enem_2014_selected = enem_2014.select(columns)
enem_2015_selected = enem_2015.select(columns)
enem_2016_selected = enem_2016.select(columns)
enem_2017_selected = enem_2017.select(columns)
enem_2018_selected = enem_2018.select(columns)
enem_2019_selected = enem_2019.select(columns)
enem_2020_selected = enem_2020.select(columns)

### Lidando com Valores NAN

In [ ]:
var_with_nan = []
def verfify_nan_values(df):
    df_nan = ps.DataFrame()
    var_without_nan = []
    var_column = []
    var_percent = []
    total = df.shape[0]
    count = 0
    for index, value in df.isna().sum().iteritems():
        if value == 0:
            var_without_nan.append(index)
        else:
            percent_with_nan = (value / total) * 100
            var_with_nan.append(count)
            var_column.append(index)
            var_percent.append(percent_with_nan)
        count  = count + 1

    d = {'coluna': var_column, 'porcentagem_nan %': var_percent}
    df_nan = ps.DataFrame(data=d)
    #df_nan.insert(0, 'coluna', var_column, True)
    #df_nan.insert(1, 'porcentagem_nan %', var_percent, True)

    return df_nan

In [ ]:
#df_nan = verfify_nan_values(enem_2020_selected)

#df_nan

### Tratamento dos Valores Ausentes

In [ ]:
enem_2019_without_nan = enem_2019_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2018_without_nan = enem_2018_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2017_without_nan = enem_2017_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2016_without_nan = enem_2016_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2015_without_nan = enem_2015_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2014_without_nan = enem_2014_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2013_without_nan = enem_2013_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2012_without_nan = enem_2012_selected.dropna(subset = ['NU_NOTA_CN'])
enem_2010_without_nan = enem_2010_selected.dropna(subset = ['NU_NOTA_CN'])

### Alterando Tipos das Variáveis

1. Variáveis Quantitativas

In [ ]:
var_quant = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q005']

In [ ]:
enem_2010_without_nan = enem_2010_without_nan.withColumn('NU_NOTA_CN',enem_2010_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2010_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2010_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2010_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2010_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2010_without_nan['Q005'].cast('int'))
enem_2012_without_nan = enem_2012_without_nan.withColumn('NU_NOTA_CN',enem_2012_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2012_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2012_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2012_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2012_without_nan['NU_NOTA_REDACAO'].cast('double'))
enem_2013_without_nan = enem_2013_without_nan.withColumn('NU_NOTA_CN',enem_2013_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2013_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2013_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2013_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2013_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2013_without_nan['Q005'].cast('int'))
enem_2014_without_nan = enem_2014_without_nan.withColumn('NU_NOTA_CN',enem_2014_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2014_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2014_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2014_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2014_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2014_without_nan['Q005'].cast('int'))
enem_2015_without_nan = enem_2015_without_nan.withColumn('NU_NOTA_CN',enem_2015_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2015_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2015_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2015_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2015_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2015_without_nan['Q005'].cast('int'))
enem_2016_without_nan = enem_2016_without_nan.withColumn('NU_NOTA_CN',enem_2016_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2016_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2016_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2016_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2016_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2016_without_nan['Q005'].cast('int'))
enem_2017_without_nan = enem_2017_without_nan.withColumn('NU_NOTA_CN',enem_2017_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2017_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2017_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2017_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2017_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2017_without_nan['Q005'].cast('int'))
enem_2018_without_nan = enem_2018_without_nan.withColumn('NU_NOTA_CN',enem_2018_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2018_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2018_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2018_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2018_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2018_without_nan['Q005'].cast('int'))
enem_2019_without_nan = enem_2019_without_nan.withColumn('NU_NOTA_CN',enem_2019_without_nan['NU_NOTA_CN'].cast('double')).withColumn('NU_NOTA_CH',enem_2019_without_nan['NU_NOTA_CH'].cast('double')).withColumn('NU_NOTA_LC',enem_2019_without_nan['NU_NOTA_LC'].cast('double')).withColumn('NU_NOTA_MT',enem_2019_without_nan['NU_NOTA_MT'].cast('double')).withColumn('NU_NOTA_REDACAO',enem_2019_without_nan['NU_NOTA_REDACAO'].cast('double')).withColumn('Q005',enem_2019_without_nan['Q005'].cast('int'))


2. Variáveis Qualitativas

In [ ]:
var_quali = ['NU_ANO', 'TP_COR_RACA', 'TP_ESTADO_CIVIL','TP_SEXO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006']

### Transformando PySpark em Pandas

### Criando Tabela Banco de Dados

### Adicionando Valores a Tabela